In [ ]:
# Install required packages (if needed)
# ! pip install llama-stack-client python-dotenv rich termcolor


In [1]:
# Import required libraries
import os
import json
from datetime import datetime
from dotenv import load_dotenv
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
from llama_stack_client.lib.agents.event_logger import EventLogger
from termcolor import cprint


In [2]:
# Load configuration
load_dotenv('config.env')

# Get configuration from environment
base_url = os.environ.get('LLAMA_STACK_URL')
model_id = os.environ.get('LLM_MODEL_ID')
temperature = float(os.environ.get("TEMPERATURE", 0.0))
max_tokens = int(os.environ.get("MAX_TOKENS", 4096))

# Sampling parameters
sampling_params = {
    "strategy": {"type": "greedy"},
    "max_tokens": max_tokens,
    "temperature": temperature,
}

print(f"✅ Configuration loaded:")
print(f"  Llama Stack URL: {base_url}")
print(f"  Model ID: {model_id}")
print(f"  Temperature: {temperature}")
print(f"  Max Tokens: {max_tokens}")


✅ Configuration loaded:
  Llama Stack URL: http://localhost:8321
  Model ID: r1-qwen-14b-w4a16
  Temperature: 0.0
  Max Tokens: 4096


In [3]:
# Initialize Llama Stack client
client = LlamaStackClient(base_url=base_url)

print(f'✅ Connected to Llama Stack server at {base_url}')
print(f'✅ Using model: {model_id}')

# Check available Atlassian MCP tools
print("\n🔍 Checking Atlassian MCP tools:")
try:
    atlassian_tools = client.tools.list(toolgroup_id="mcp::atlassian")
    print(f"✅ Found {len(atlassian_tools)} Atlassian tools")
    
    # Find Jira tools
    jira_tools = [t for t in atlassian_tools if 'jira' in str(t).lower()]
    print(f"🎫 Jira tools: {len(jira_tools)}")
    for tool in jira_tools:
        tool_name = getattr(tool, 'name', getattr(tool, 'identifier', str(tool)))
        print(f"   - {tool_name}")
        
except Exception as e:
    print(f"❌ Error loading tools: {e}")


✅ Connected to Llama Stack server at http://localhost:8321
✅ Using model: r1-qwen-14b-w4a16

🔍 Checking Atlassian MCP tools:
✅ Found 37 Atlassian tools
🎫 Jira tools: 27
   - jira_add_comment
   - jira_add_worklog
   - jira_batch_create_issues
   - jira_batch_get_changelogs
   - jira_create_issue
   - jira_create_issue_link
   - jira_create_sprint
   - jira_delete_issue
   - jira_download_attachments
   - jira_get_agile_boards
   - jira_get_board_issues
   - jira_get_issue
   - jira_get_link_types
   - jira_get_project_issues
   - jira_get_project_versions
   - jira_get_sprint_issues
   - jira_get_sprints_from_board
   - jira_get_transitions
   - jira_get_user_profile
   - jira_get_worklog
   - jira_link_to_epic
   - jira_remove_issue_link
   - jira_search
   - jira_search_fields
   - jira_transition_issue
   - jira_update_issue
   - jira_update_sprint


In [4]:
# Create Jira incident creation agent
jira_agent = ReActAgent(
    client=client,
    model=model_id,
    instructions="""You are a helpful assistant that can create Jira incidents and tickets. 
    Use the available Jira tools to create, update, and manage issues in the KAN project.
    When creating incidents, provide detailed descriptions and use appropriate issue types.
    Always use the correct dictionary format for additional_fields, not JSON strings.""",
    tools=["mcp::atlassian"],
    response_format={
        "type": "json_schema",
        "json_schema": ReActOutput.model_json_schema(),
    },
    sampling_params=sampling_params,
)

print("✅ Jira incident creation agent created")


✅ Jira incident creation agent created


In [5]:
# Define OOM error incident details
oom_incident = {
    "project_key": "KAN",
    "summary": "Pod failing due to OOM error",
    "issue_type": "Incident",
    "description": """## Pod Out of Memory (OOM) Error

**Severity**: High
**Priority**: P1
**Environment**: Production/Staging
**Timestamp**: {timestamp}

### Problem Description
Pod is experiencing Out of Memory (OOM) errors causing application failures and service disruption.

### Symptoms
- Pod termination due to memory limits
- Application crashes
- Service unavailability
- Error logs showing OOMKilled status

### Impact
- Service downtime
- User experience degradation
- Potential data loss
- Business operations affected

### Root Cause Analysis
- Memory usage exceeding pod limits
- Possible memory leaks in application
- Insufficient resource allocation
- High memory consumption during peak loads

### Immediate Actions Required
1. Scale up pod resources if possible
2. Investigate memory usage patterns
3. Check for memory leaks in application code
4. Monitor resource utilization
5. Implement proper resource limits

### Additional Information
- Pod Name: [To be filled]
- Namespace: [To be filled]
- Node: [To be filled]
- Memory Limit: [To be filled]
- Current Memory Usage: [To be filled]

### Labels
- oom-error
- pod-failure
- memory-issue
- high-priority
- production-incident
""".format(timestamp=datetime.now().strftime("%Y-%m-%d %H:%M:%S")),
    "additional_fields": {
        "priority": {"name": "High"},
        "labels": ["oom-error", "pod-failure", "memory-issue", "high-priority"],
        "components": ["Infrastructure", "Kubernetes"],
        "fixVersions": ["TBD"],
        "assignee": "TBD"
    }
}

print("📋 OOM Error Incident Details:")
print(f"  Project: {oom_incident['project_key']}")
print(f"  Summary: {oom_incident['summary']}")
print(f"  Issue Type: {oom_incident['issue_type']}")
print(f"  Priority: {oom_incident['additional_fields']['priority']['name']}")
print(f"  Labels: {', '.join(oom_incident['additional_fields']['labels'])}")


📋 OOM Error Incident Details:
  Project: KAN
  Summary: Pod failing due to OOM error
  Issue Type: Incident
  Priority: High
  Labels: oom-error, pod-failure, memory-issue, high-priority


In [6]:
# Function to create Jira incident
def create_jira_incident(incident_details):
    """Create a Jira incident with the provided details"""
    print(f"\n🎫 Creating Jira incident: '{incident_details['summary']}'")
    print("="*60)
    
    try:
        # Create a session
        session_id = jira_agent.create_session("jira-oom-incident")
        print(f"✅ Session created: {session_id}")
        
        # Create the incident creation prompt
        incident_prompt = f"""
Create a Jira incident in the {incident_details['project_key']} project with these parameters:

- project_key: "{incident_details['project_key']}"
- summary: "{incident_details['summary']}"
- issue_type: "{incident_details['issue_type']}"
- description: "{incident_details['description']}"
- additional_fields: {incident_details['additional_fields']}

Use the correct dictionary format for additional_fields, not a JSON string.
Make sure to use the exact values provided above.
"""
        
        print(f"\n📝 Incident Creation Prompt:")
        print(incident_prompt)
        print("\n" + "="*60)
        
        # Create a turn with the incident creation request
        response = jira_agent.create_turn(
            messages=[{"role": "user", "content": incident_prompt}],
            session_id=session_id,
            stream=True,
            toolgroups=["mcp::atlassian"]  # Explicit toolgroups list
        )
        
        print("\n🚀 Processing incident creation...")
        
        # Process the streaming response
        for log in EventLogger().log(response):
            log.print()
            
        return session_id
        
    except Exception as e:
        print(f"❌ Error creating incident: {e}")
        return None

# Create the OOM error incident
incident_session = create_jira_incident(oom_incident)



🎫 Creating Jira incident: 'Pod failing due to OOM error'
✅ Session created: dd03c415-374c-4616-b670-41053a6968cc

📝 Incident Creation Prompt:

Create a Jira incident in the KAN project with these parameters:

- project_key: "KAN"
- summary: "Pod failing due to OOM error"
- issue_type: "Incident"
- description: "## Pod Out of Memory (OOM) Error

**Severity**: High
**Priority**: P1
**Environment**: Production/Staging
**Timestamp**: 2025-09-29 07:43:22

### Problem Description
Pod is experiencing Out of Memory (OOM) errors causing application failures and service disruption.

### Symptoms
- Pod termination due to memory limits
- Application crashes
- Service unavailability
- Error logs showing OOMKilled status

### Impact
- Service downtime
- User experience degradation
- Potential data loss
- Business operations affected

### Root Cause Analysis
- Memory usage exceeding pod limits
- Possible memory leaks in application
- Insufficient resource allocation
- High memory consumption during 

In [7]:
# Alternative method: Direct tool call (if agent method fails)
def create_incident_direct():
    """Create incident using direct tool calls"""
    print("\n🔧 Attempting direct tool call method...")
    
    try:
        # Get available tools
        atlassian_tools = client.tools.list(toolgroup_id="mcp::atlassian")
        
        # Find Jira create issue tool
        jira_create_tool = None
        for tool in atlassian_tools:
            tool_name = getattr(tool, 'name', getattr(tool, 'identifier', str(tool)))
            if 'jira' in str(tool).lower() and 'create' in str(tool).lower():
                jira_create_tool = tool_name
                break
        
        if jira_create_tool:
            print(f"✅ Found Jira create tool: {jira_create_tool}")
            
            # Try to call it directly
            result = client.tools.call(
                tool_name=jira_create_tool,
                tool_params=[
                    {"name": "project_key", "value": oom_incident['project_key']},
                    {"name": "summary", "value": oom_incident['summary']},
                    {"name": "issue_type", "value": oom_incident['issue_type']},
                    {"name": "description", "value": oom_incident['description']},
                    {"name": "additional_fields", "value": oom_incident['additional_fields']}
                ]
            )
            print(f"✅ Direct tool call successful!")
            print(f"Result: {result}")
            return result
        else:
            print("❌ No Jira create tool found")
            return None
            
    except Exception as e:
        print(f"❌ Direct tool call failed: {e}")
        return None

# Uncomment to try direct method
# create_incident_direct()


In [8]:
# Function to verify incident creation
def verify_incident_creation(session_id):
    """Verify that the incident was created successfully"""
    if not session_id:
        print("❌ No session ID provided for verification")
        return False
    
    print(f"\n🔍 Verifying incident creation for session: {session_id}")
    
    try:
        # Get session details
        session_details = jira_agent.client.agents.session.retrieve(
            agent_id=jira_agent.agent_id,
            session_id=session_id
        )
        
        print(f"✅ Session details retrieved")
        print(f"  Session Name: {session_details.session_name}")
        print(f"  Created At: {session_details.created_at}")
        
        # Get turn details
        turns = jira_agent.client.agents.turn.list(
            agent_id=jira_agent.agent_id,
            session_id=session_id
        )
        
        print(f"✅ Found {len(turns)} turns in session")
        
        for i, turn in enumerate(turns):
            print(f"\n📝 Turn {i+1}:")
            print(f"  Turn ID: {turn.turn_id}")
            print(f"  Created At: {turn.created_at}")
            
            # Get turn details
            turn_details = jira_agent.client.agents.turn.retrieve(
                agent_id=jira_agent.agent_id,
                session_id=session_id,
                turn_id=turn.turn_id
            )
            
            print(f"  Status: {turn_details.status}")
            if hasattr(turn_details, 'messages') and turn_details.messages:
                print(f"  Messages: {len(turn_details.messages)}")
                for msg in turn_details.messages:
                    if hasattr(msg, 'role') and hasattr(msg, 'content'):
                        print(f"    {msg.role}: {msg.content[:100]}...")
        
        return True
        
    except Exception as e:
        print(f"❌ Error verifying incident: {e}")
        return False

# Verify the incident creation
if incident_session:
    verify_incident_creation(incident_session)
else:
    print("❌ No session ID available for verification")



🔍 Verifying incident creation for session: dd03c415-374c-4616-b670-41053a6968cc
✅ Session details retrieved
  Session Name: jira-oom-incident
❌ Error verifying incident: 'Session' object has no attribute 'created_at'
